In [ ]:
from flask import Flask, render_template, request
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
import pickle
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
import better_profanity

app = Flask(__name__)

# Load the sentiment analysis model and tokenizer
loaded_model = tf.keras.models.load_model("D:\pythonana\model.h5")
with open(r"D:\pythonana\tokenizer.pkl", 'rb') as f:
    tokenizer = pickle.load(f)
with open(r'D:\pythonana\label_encoder.pkl', 'rb') as f:
    label_encoder = pickle.load(f)

# Load the suggestion analysis model and tokenizer
suggestion_model = tf.keras.models.load_model("D:\pythonana\suggestionmodel.h5")
with open(r"D:\pythonana\suggestiontokenizer.pkl", 'rb') as f:
    suggestion_tokenizer = pickle.load(f)
with open(r"D:\pythonana\suggestion_label_encoder.pkl", 'rb') as f:
    suggestion_label_encoder = pickle.load(f)

# Create an instance of the SentimentIntensityAnalyzer for the overall review
sid = SentimentIntensityAnalyzer()

# Download the VADER lexicon if not already downloaded
# nltk.download('vader_lexicon')


def assign_emoji(sentiment_percentage):
    if sentiment_percentage >= 70:
        emoji = '😃'  # Very positive emoji
    elif sentiment_percentage >= 40:
        emoji = '🙂'  # Positive emoji
    elif sentiment_percentage >= 10:
        emoji = '😐'  # Neutral emoji
    elif sentiment_percentage >= -30:
        emoji = '😔'  # Negative emoji
    else:
        emoji = '😡'  # Very negative emoji
    return emoji

def get_sentiment_percentage(sentence, sentiment):
    sentiment_scores = sid.polarity_scores(sentence)
    if sentiment == 'positive':
        sentiment_score = sentiment_scores['pos']
        sentiment_percentage = sentiment_score * 100
        emoji = assign_emoji(sentiment_percentage)
    elif sentiment == 'negative':
        sentiment_score = sentiment_scores['neg']
        sentiment_percentage = sentiment_score * 100
        emoji = assign_emoji(100 - 31 - sentiment_percentage)
    elif sentiment == 'neutral':
        sentiment_score = sentiment_scores['neu']
        sentiment_percentage = sentiment_score * 100
        emoji = assign_emoji(50 - 11 - sentiment_percentage)
    else:
        raise ValueError("Invalid sentiment provided. Must be 'positive', 'negative', or 'neutral'.")
    return f"{emoji} {sentiment_percentage:.2f}%"


def check_offensive_content(text):
    return better_profanity.profanity.contains_profanity(text)


# Home page
@app.route('/', methods=['GET'])
@app.route('/home', methods=['GET'])
def home():
    return render_template('home.html')

@app.route('/index', methods=['GET', 'POST'])
def sentiment_analysis():
    if request.method == 'POST':
        sentence = request.form.get('sentence')
        sentiment = request.form.get('sentiment')
        resul = get_sentiment_percentage(sentence, sentiment)
        return render_template('index.html', result=[resul,sentence,sentiment])
    
    # Handle the GET request
    return render_template('index.html')



# Sentiment Analysis page
@app.route('/sentimentindex', methods=['GET', 'POST'])
def sentiment_analyze():
    if request.method == 'POST':
        # Get user input from the form
        incoming_text = request.form['input_text']

        # Preprocess the incoming text
        incoming_sequence = tokenizer.texts_to_sequences([incoming_text])
        incoming_padded_sequence = pad_sequences(incoming_sequence, maxlen=65)

        # Predict the sentiment label for the incoming data
        predicted_label = np.argmax(loaded_model.predict(incoming_padded_sequence), axis=-1)

        # Decode the predicted label
        decoded_label = label_encoder.inverse_transform(predicted_label)[0]

        # Return the prediction as the response
        return render_template('sentimentindex.html', prediction=decoded_label, input_text=incoming_text)

    return render_template('sentimentindex.html')




# Fake Review Detection page
@app.route('/fakereviewindex', methods=['GET', 'POST'])
def fake_review_detection():
    if request.method == 'POST':
        text = request.form.get('text')
        is_offensive = check_offensive_content(text)

        if is_offensive:
            result = f"'{text}' contains offensive content."
        else:
            result = f"'{text}' is clean."
        
        return render_template('fakereviewindex.html', result=result)
    
    return render_template('fakereviewindex.html')




# Suggestion Analysis page
@app.route('/suggestionindex', methods=['GET', 'POST'])
def suggestion_analysis():

    if request.method == 'POST':
        incoming_text = request.form['text_input']

    # Preprocess the incoming text
        if incoming_text is None:
        # Handle the case when the input text is not provided
        # You can redirect the user back to the form or display an error message
            return render_template('suggestionindex.html')
        incoming_sequence = suggestion_tokenizer.texts_to_sequences([incoming_text])
        incoming_padded_sequence = pad_sequences(incoming_sequence, maxlen=9)

        # Predict the sentiment label for the incoming data
        predicted_label = np.argmax(suggestion_model.predict(incoming_padded_sequence), axis=-1)
#         print(predicted_label)
        if predicted_label >= 0 and predicted_label < len(suggestion_label_encoder.classes_):
            decoded_label = suggestion_label_encoder.classes_[predicted_label]
        else:
            decoded_label = 'Suggestion'  # Assign a default label for unseen predictions

        # Decode the predicted label
        jik=[incoming_text,decoded_label]
        return render_template('suggestionindex.html', prediction=jik)
    
    return render_template('suggestionindex.html')




try:
    if __name__ == '__main__':
        app.run()
except Exception as e:
    print(e)

